In [1]:
import re
import dash
import json
import numpy as np
import pandas as pd
import plotly.express as px
from dash import dcc
from dash import html
from dash import dash_table
from datetime import datetime 
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']



/home/luchoss10/.local/lib/python3.8/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [26]:
def getSheetData (df):
    pollos_erroneos = 0
    datos_erroneos = 0
    fecha =  df.iloc[2,1]
    semana = df.iloc[3, 1]
    #print(semana)
    #Dataframe para concatenar los datos de cada granja en el sheet
    sheet_df = pd.DataFrame()
    sheet_df2 = pd.DataFrame()

    #Creacion de primeras columnas 
    frist_cols = df.iloc[6:,1:3]
    first_cols_names = ['lesionTipo', 'lesionRango']
    frist_cols.columns = first_cols_names
    #list_ids = ['1','2','3','4','5']

    for i, val in enumerate(range(3, len(df.columns),6)):
        list_ids = ['1','2','3','4','5']
        #Informacion para transformar 
        content = df.iloc[6:, val:val+5]
        content.columns = list_ids
        content = content.dropna(axis='columns', how = 'all')
        list_ids = content.columns

        #Valores individuales
        granja = df.iloc[0,val]

        if pd.isnull(granja):
            pass
        else:
            #fecha = datetime.fromtimestamp(fecha).strftime('%y-%m-%d')
            edad_str = str(df.iloc[1,val]).replace(' ', '')
            #print(edad_str)
            edad_dias = int(re.findall(r'\d+', edad_str)[0]) # cambio 
            ciclo = float(df.iloc[2,val])
            no_galpon = float(df.iloc[4,val])
            planVacuna = df.iloc[0,val+3]
            influenzaVacuna = df.iloc[1,val+3]
            newcastleVacuna = df.iloc[2,val+3]
            gumboroVacuna = df.iloc[3,val+3]

            #Union rangos, lesiones con valores
            wdf = pd.concat([frist_cols, content], axis=1)
            #Cambio de valores Nulos por 0
            wdf.fillna(0, inplace=True)
            wdf.replace('', 0)
            wdf.replace(' ', 0)
            wdf.replace('  ', 0)
            wdf.replace('   ', 0)

            # --- Verificacion de valores de lesion en los rangos correspondientes ---
            #Verificacion de rango en string para la verificacion
            wdf.astype({first_cols_names[1]: 'str'})
            wdf2 = wdf
            for row, index in enumerate(wdf.index):
                cuenta = 0
                entrar = True
                lesion = wdf.iloc[row][first_cols_names[0]]
                if (lesion != 'Sexo' and 
                    lesion != 'Peso' and 
                    lesion != 'Condición de Hígado' and 
                    lesion !='INTEGRIDAD INTESTINAL'):
                    rango = str(wdf.iloc[row][first_cols_names[1]]).split('-')
                    for col in list_ids:
                        valor =  wdf.iloc[row][col]
                        if valor >= int(rango[0]) and valor <= int(rango[1]):
                            pass
                        else:
                            datos_erroneos += 1
                            wdf.loc[index, col] = int(rango[0])
                            if entrar:
                                cuenta += 1
                                entrar = False
                
                if lesion == 'Sexo':
                    rango = wdf.iloc[row][first_cols_names[1]].split('/')
                    for col in list_ids:
                        valor = wdf.iloc[row][col]
                        if (valor is rango[0]) or (valor is rango[1]) or (valor == 0):
                            pass
                        else:
                            datos_erroneos += 1
                            wdf.loc[index, col] = 0
                            if entrar:
                                cuenta += 1
                                entrar = False

                if lesion == 'Condición de Hígado':
                    rango = wdf.iloc[row][first_cols_names[1]].split('-')
                    for col in list_ids:
                        valor = wdf.iloc[row][col]
                        if isinstance(valor, str):
                            if ((rango[0] in valor) or 
                                (rango[1] in valor) or 
                                (rango[2] in valor)):
                                pass
                            else:
                                datos_erroneos += 1
                                wdf.loc[index, col] = 0
                                if entrar:
                                    cuenta += 1
                                    entrar = False
                        elif valor == 0:
                            pass
                        else:
                            datos_erroneos += 1
                            wdf.loc[index, col] = 0
                            if entrar:
                                cuenta += 1
                                entrar = False
                
                pollos_erroneos =  pollos_erroneos + cuenta
                    
            
            wdf = pd.melt(wdf, id_vars=[first_cols_names[0], first_cols_names[1]], value_vars=list_ids, 
                            var_name='nAnimal', value_name='lesionPromedio')

            wdf2 =  pd.melt(wdf2, id_vars=[first_cols_names[0], first_cols_names[1]], value_vars=list_ids, 
                            var_name='nAnimal', value_name='lesionPromedio')

            #Mapeo a nombre de columnas finales
            map_etiqueta = {'Sexo' : 'sexoAnimales', 
                            'Bursometro' : 'bursometro', 
                            'Condición de Hígado' : 'condicionHigado', 
                            'INTEGRIDAD INTESTINAL' : 'integridadintestinal'}
            
            for key in map_etiqueta:
                wdf[map_etiqueta[key]] = ''
                values = wdf[wdf[first_cols_names[0]] == key]
                wdf2[map_etiqueta[key]] = ''
                values2 = wdf2[wdf2[first_cols_names[0]] == key]
                for j in range(1, len(values.index)+1):
                    idx = values.index[j-1]
                    idx2 = values2.index[j-1]
                    wdf.drop(idx, inplace=True)
                    wdf.loc[wdf.nAnimal == str(j), map_etiqueta[key]] = values.iloc[j-1,3]
                    wdf2.drop(idx2, inplace=True)
                    wdf2.loc[wdf2.nAnimal == str(j), map_etiqueta[key]] = values2.iloc[j-1,3]

            wdf['fecha'] = fecha
            wdf['semana'] = semana
            wdf['granja'] = granja
            wdf['edadEnDias'] = edad_dias
            wdf['ciclo'] = ciclo
            wdf['noGalpon'] = no_galpon
            wdf['planVacuna'] = planVacuna
            wdf['influenzaVacuna'] = influenzaVacuna
            wdf['newcastleVacuna'] = newcastleVacuna
            wdf['gumboroVacuna'] = gumboroVacuna

            wdf2['fecha'] = fecha
            wdf2['semana'] = semana
            wdf2['granja'] = granja
            wdf2['edadEnDias'] = edad_dias
            wdf2['ciclo'] = ciclo
            wdf2['noGalpon'] = no_galpon
            wdf2['planVacuna'] = planVacuna
            wdf2['influenzaVacuna'] = influenzaVacuna
            wdf2['newcastleVacuna'] = newcastleVacuna
            wdf2['gumboroVacuna'] = gumboroVacuna

            sheet_df = sheet_df.append(wdf)
            sheet_df2 = sheet_df2.append(wdf2)
            
    return sheet_df, sheet_df2, datos_erroneos, pollos_erroneos

In [11]:
sheets = list(range(1,   35))
dict_data = pd.read_excel('prueba.xlsx', sheet_name=sheets, header=None)
with open('aux_data.json') as f:
    aux_data = json.load(f)

In [27]:
datos_errados = 0
pollos_e = 0
clean_complete_data = pd.DataFrame()
all_complete_data = pd.DataFrame()
for key in dict_data:
     df = dict_data[key].iloc[:44,:]
     sheet_df, sheet_df2, datos_erroneos, pollos_erroneos = getSheetData(df)
     clean_complete_data = clean_complete_data.append(sheet_df)
     all_complete_data = all_complete_data.append(sheet_df2)

     datos_errados = datos_erroneos + datos_errados 
     pollos_e = pollos_e + pollos_erroneos

clean_complete_data = clean_complete_data.loc[clean_complete_data['lesionTipo'] != 'Peso']
all_complete_data = all_complete_data.loc[all_complete_data['lesionTipo'] != 'Peso']

clean_complete_data = clean_complete_data.reset_index(drop=True)
all_complete_data = all_complete_data.reset_index(drop=True)


clean_complete_data.loc[clean_complete_data.granja == 'Fortuna ', 'granja'] = 'Fortuna'
clean_complete_data.loc[clean_complete_data.granja == 'Vergel ', 'granja'] = 'Vergel'
clean_complete_data.loc[clean_complete_data.granja == 'Sta. Delfina', 'granja'] = 'Santa Defina'

all_complete_data.loc[all_complete_data.granja == 'Fortuna ', 'granja'] = 'Fortuna'
all_complete_data.loc[all_complete_data.granja == 'Vergel ', 'granja'] = 'Vergel'
all_complete_data.loc[all_complete_data.granja == 'Sta. Delfina', 'granja'] = 'Santa Defina'

clean_complete_data['fecha'] = pd.to_datetime(clean_complete_data['fecha'])
all_complete_data['fecha'] = pd.to_datetime(all_complete_data['fecha'])
#print(complete_data.head())

In [28]:
print(datos_errados)

print(pollos_e)

79
68


In [6]:
def date_string_to_date(date_string):
    return pd.to_datetime(date_string, infer_datetime_format=True)

In [7]:
#consulta2 = clean_complete_data.loc[(clean_complete_data['semana'] == 16) & (clean_complete_data['granja'] == 'Vergel')]
#clean_complete_data['granja'].unique()
#all_complete_data['granja'].unique()

cosa = clean_complete_data.loc[:2]
#cosa['fecha'] = cosa['fecha'].dt.date
cosa.dtypes['fecha']
#cosa


dtype('<M8[ns]')

In [8]:
data = pd.read_csv('lesiones_pollos.csv')
data = data.astype({'fecha':'datetime64[ns]'}, errors='ignore')
#print(type(data['fecha'][0]))

In [9]:
lesi_drop_list = data['lesionTipo'].unique()
lesi_drop_list = np.insert(lesi_drop_list, 0, 'Todas', axis=0)
gran_drop_list = data['granja'].unique()
gran_drop_list =  np.insert(gran_drop_list, 0, 'Todas', axis=0)

first_date = data.fecha.min()
print(first_date)
lasT_date = data.fecha.max()
print(lasT_date)
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

#clean_complete_data.loc[clean_complete_data['fecha'] >= first_date]

PAGESIZE = 33

app.layout = dbc.Container(
    [
        html.H1("Información Lesiones Pollos"),
        html.Hr(),
        dash_table.DataTable(id='table_grid', 
                            columns=[
                                {'name': i ,'id':i} for i in clean_complete_data.columns
                                ],
                                page_current= 0,
                                page_size=PAGESIZE,
                                page_action='custom'
                            ),
        dbc.FormGroup(
            [
                dbc.Label("Granja"),
                dcc.Dropdown(
                    id='dGranja',
                    value='Todas',
                    clearable=False,
                    multi=False,
                    disabled=False,
                    searchable=True,
                    persistence='string',
                    persistence_type='local',
                    options= [
                        {'label': i, 'value':i}
                        for i in gran_drop_list
                    ],
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Lesión"),
                dcc.Dropdown(
                    id='dLesion',
                    value='Todas',
                    clearable=False,
                    multi=False,
                    disabled=False,
                    searchable=True,
                    persistence='string',
                    persistence_type='local',
                    options=[
 
                        {'label': i, 'value':i}
                        for i in lesi_drop_list
                    ],
                ),
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Rango De Fechas"),
                dbc.FormGroup(
                    dcc.DatePickerRange(
                        id = 'date_range',
                        min_date_allowed=first_date,
                        max_date_allowed=lasT_date,
                        start_date=first_date,
                        end_date=lasT_date
                    )
                )
            ]
        ),
    ],
    fluid=True,
)

@app.callback(
    Output('table_grid', 'data'),
    [
        Input('table_grid', 'page_current'),
        Input('table_grid', 'page_size'),
        Input('dGranja', 'value'),
        Input('dLesion', 'value'),
        Input('date_range', 'start_date'),
        Input('date_range', 'end_date')
    ]
)

def update_table(page_current, page_size, granja, lesion, start_date, end_date):
    
    if lesion == 'Todas' and granja == 'Todas':
        consulta = data.loc[(data['fecha'] >= start_date) & (data['fecha'] <= end_date)]
    
    elif lesion == 'Todas' and granja != 'Todas':
        consulta = data.loc[(data['granja'] == granja) & (data['fecha'] >= start_date) & (data['fecha'] <= end_date)]
    elif lesion != 'Todas' and granja == 'Todas':
        consulta = data.loc[(data['lesionTipo'] == lesion)& (data['fecha'] >= start_date) & (data['fecha'] <= end_date)]
    else:
        consulta = data.loc[(data['granja'] == granja) & (data['lesionTipo'] == lesion) & (data['fecha'] >= start_date) & (data['fecha'] <= end_date)]
        
    consulta.reset_index(drop=True, inplace=True)
    consulta = consulta.iloc[page_current*page_size:(page_current+1)*page_size]

    return consulta.to_dict('records')
    

app.run_server(mode='inline')
#if __name__ == '__main__':
#    app.run_server(debug=False)

2020-01-07 00:00:00
2020-12-30 00:00:00
